### 1. 安装HuggingFace 并下载模型到本地

In [70]:
!pip install huggingface-hub -Uqq -i https://pypi.tuna.tsinghua.edu.cn/simple/
!pip install -U sagemaker -i https://pypi.tuna.tsinghua.edu.cn/simple/
!pip install --upgrade sagemaker -i https://pypi.tuna.tsinghua.edu.cn/simple/

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple/, https://pip.repos.neuron.amazonaws.com
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple/, https://pip.repos.neuron.amazonaws.com


In [27]:
from huggingface_hub import snapshot_download
from pathlib import Path

local_model_path = Path("./sentence2emb-model")
local_model_path.mkdir(exist_ok=True)
model_name = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
commit_hash = "ef15aed8b328d308d7237b9bf15269f2cd19e268"

In [28]:
#执行这段，因为网络原因，可能会多次失败，需要反复执行
while True:
    try:
        snapshot_download(repo_id=model_name, revision=commit_hash, cache_dir=local_model_path)
        break
    except Exception:
        pass

Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

### 2. 把模型拷贝到S3为后续部署做准备

In [49]:
import sagemaker
from sagemaker import image_uris
import boto3
import os
import time
import json

role = sagemaker.get_execution_role()  # execution role for the endpoint
sess = sagemaker.session.Session()  # sagemaker session for interacting with different AWS APIs
bucket = sess.default_bucket()  # bucket to house artifacts

region = sess._region_name
account_id = sess.account_id()

s3_client = boto3.client("s3")
sm_client = boto3.client("sagemaker")
smr_client = boto3.client("sagemaker-runtime")

In [50]:
s3_model_prefix = "LLM-RAG/workshop/sentence2emb-model"  # folder where model checkpoint will go
model_snapshot_path = list(local_model_path.glob("**/snapshots/*"))[0]
s3_code_prefix = "LLM-RAG/workshop/sentence2emb_deploy_code"
print(f"s3_code_prefix: {s3_code_prefix}")
print(f"model_snapshot_path: {model_snapshot_path}")

s3_code_prefix: LLM-RAG/workshop/sentence2emb_deploy_code
model_snapshot_path: sentence2emb-model/models--sentence-transformers--paraphrase-multilingual-mpnet-base-v2/snapshots/ef15aed8b328d308d7237b9bf15269f2cd19e268


In [52]:
#上传模型至S3
!aws s3 cp --recursive {model_snapshot_path} s3://{bucket}/{s3_model_prefix}

upload: sentence2emb-model/models--sentence-transformers--paraphrase-multilingual-mpnet-base-v2/snapshots/ef15aed8b328d308d7237b9bf15269f2cd19e268/README.md to s3://sagemaker-cn-north-1-507392672631/LLM-RAG/workshop/sentence2emb-model/README.md
upload: sentence2emb-model/models--sentence-transformers--paraphrase-multilingual-mpnet-base-v2/snapshots/ef15aed8b328d308d7237b9bf15269f2cd19e268/.gitattributes to s3://sagemaker-cn-north-1-507392672631/LLM-RAG/workshop/sentence2emb-model/.gitattributes
upload: sentence2emb-model/models--sentence-transformers--paraphrase-multilingual-mpnet-base-v2/snapshots/ef15aed8b328d308d7237b9bf15269f2cd19e268/modules.json to s3://sagemaker-cn-north-1-507392672631/LLM-RAG/workshop/sentence2emb-model/modules.json
upload: sentence2emb-model/models--sentence-transformers--paraphrase-multilingual-mpnet-base-v2/snapshots/ef15aed8b328d308d7237b9bf15269f2cd19e268/config_sentence_transformers.json to s3://sagemaker-cn-north-1-507392672631/LLM-RAG/workshop/sentence2

### 3. 模型部署准备（entrypoint脚本，容器镜像，服务配置）

In [53]:
#适用于global
#inference_image_uri = (
#    f"763104351884.dkr.ecr.{region}.amazonaws.com/djl-inference:0.21.0-deepspeed0.8.3-cu117"
#)

#中国区需要替换为下面的image_uri
inference_image_uri = (
     f"727897471807.dkr.ecr.{region}.amazonaws.com.cn/djl-inference:0.21.0-deepspeed0.8.3-cu117"
)

print(f"Image going to be used is ---- > {inference_image_uri}")

Image going to be used is ---- > 727897471807.dkr.ecr.cn-north-1.amazonaws.com.cn/djl-inference:0.21.0-deepspeed0.8.3-cu117


In [12]:
!mkdir -p sentence2emb_deploy_code

In [73]:
%%writefile sentence2emb_deploy_code/model.py
from djl_python import Input, Output
import torch
import logging
import math
import os
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer, AutoModel

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f'--device={device}')


def load_model(properties):
    tensor_parallel = properties["tensor_parallel_degree"]
    model_location = properties['model_dir']
    if "model_id" in properties:
        model_location = properties['model_id']
    logging.info(f"Loading model in {model_location}")
    
    tokenizer = AutoTokenizer.from_pretrained(model_location)
   
    model = AutoModel.from_pretrained(
        model_location, 
        # device_map="balanced_low_0", 
        #load_in_8bit=True
    )
    # load the model on GPU
    model.to(device) 
    model.requires_grad_(False)
    model.eval()
    
    return model, tokenizer


model = None
tokenizer = None
generator = None

def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0].to(device) #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float().to(device)
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


def handle(inputs: Input):
    global model, tokenizer
    if not model:
        model, tokenizer = load_model(inputs.get_properties())

    if inputs.is_empty():
        return None
    data = inputs.get_as_json()
    
    input_sentences = data["inputs"]
    #params = data["parameters"]
    logging.info(f"inputs: {input_sentences}")
    #logging.info(f"parameters: {params}")
    
    encoded_input = tokenizer(input_sentences, padding=True, truncation=True, max_length=128, return_tensors='pt').to(device)
    # Compute token embeddings
    with torch.no_grad():
        model_output = model(**encoded_input)

    # Perform pooling. In this case, max pooling.
    sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask']).to(device).cpu().numpy()

#     # preprocess
#     input_ids = tokenizer(input_sentences, return_tensors="pt").input_ids
#     # pass inputs with all kwargs in data
#     if params is not None:
#         outputs = model.generate(input_ids, **params)
#     else:
#         outputs = model.generate(input_ids)

#     # postprocess the prediction
#     prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    result = {"sentence_embeddings": sentence_embeddings}
    return Output().add_as_json(result)

Overwriting sentence2emb_deploy_code/model.py


In [55]:
print(f"option.s3url ==> s3://{bucket}/{s3_model_prefix}/")

#使用打印出来的URL，替换下面一段代码中的S3Url

option.s3url ==> s3://sagemaker-cn-north-1-507392672631/LLM-RAG/workshop/sentence2emb-model/


#### Note: option.s3url 需要按照自己的账号进行修改, 可以拷贝上一个cell的输出

In [74]:
%%writefile sentence2emb_deploy_code/serving.properties
engine=Python
option.tensor_parallel_degree=1
option.s3url = s3://sagemaker-cn-north-1-507392672631/LLM-RAG/workshop/sentence2emb-model/

Overwriting sentence2emb_deploy_code/serving.properties


In [75]:
!rm s2e_model.tar.gz
!cd sentence2emb_deploy_code && rm -rf ".ipynb_checkpoints"
!tar czvf s2e_model.tar.gz sentence2emb_deploy_code

sentence2emb_deploy_code/
sentence2emb_deploy_code/serving.properties
sentence2emb_deploy_code/model.py


In [76]:
s3_code_artifact = sess.upload_data("s2e_model.tar.gz", bucket, s3_code_prefix)
print(f"S3 Code or Model tar ball uploaded to --- > {s3_code_artifact}")

S3 Code or Model tar ball uploaded to --- > s3://sagemaker-cn-north-1-507392672631/LLM-RAG/workshop/sentence2emb_deploy_code/s2e_model.tar.gz


### 4. 创建模型 & 创建endpoint

In [77]:
from sagemaker.utils import name_from_base
import boto3

model_name ='huggingface-inference-eb' # name_from_base("st-paraphrase-mpnet-base-v2") Note: Need to specify model_name
print(model_name)
print(f"Image going to be used is ---- > {inference_image_uri}")

create_model_response = sm_client.create_model(
    ModelName=model_name,
    ExecutionRoleArn=role,
    PrimaryContainer={
        "Image": inference_image_uri,
        "ModelDataUrl": s3_code_artifact
    },
    
)
model_arn = create_model_response["ModelArn"]

print(f"Created Model: {model_arn}")

huggingface-inference-eb
Image going to be used is ---- > 727897471807.dkr.ecr.cn-north-1.amazonaws.com.cn/djl-inference:0.21.0-deepspeed0.8.3-cu117
Created Model: arn:aws-cn:sagemaker:cn-north-1:507392672631:model/huggingface-inference-eb


In [78]:
endpoint_config_name =model_name  # f"{model_name}-config"
endpoint_name = model_name # f"{model_name}-endpoint"

endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "VariantName": "variant1",
            "ModelName": model_name,
            "InstanceType": "ml.g4dn.2xlarge",
            "InitialInstanceCount": 1,
            # "VolumeSizeInGB" : 400,
            # "ModelDataDownloadTimeoutInSeconds": 2400,
            #"ContainerStartupHealthCheckTimeoutInSeconds": 15*60,
        },
    ],
)
endpoint_config_response

{'EndpointConfigArn': 'arn:aws-cn:sagemaker:cn-north-1:507392672631:endpoint-config/huggingface-inference-eb',
 'ResponseMetadata': {'RequestId': '5ea62cd9-f9eb-4c7a-81c5-44ffdb0a2469',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '5ea62cd9-f9eb-4c7a-81c5-44ffdb0a2469',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '109',
   'date': 'Thu, 20 Jul 2023 05:06:06 GMT'},
  'RetryAttempts': 0}}

In [79]:
create_endpoint_response = sm_client.create_endpoint(
    EndpointName=f"{endpoint_name}", EndpointConfigName=endpoint_config_name
)
print(f"Created Endpoint: {create_endpoint_response['EndpointArn']}")

Created Endpoint: arn:aws-cn:sagemaker:cn-north-1:507392672631:endpoint/huggingface-inference-eb


In [80]:
import time

resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp["EndpointStatus"]
print("Status: " + status)

while status == "Creating":
    time.sleep(60)
    resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
    status = resp["EndpointStatus"]
    print("Status: " + status)

print("Arn: " + resp["EndpointArn"])
print("Status: " + status)

Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: InService
Arn: arn:aws-cn:sagemaker:cn-north-1:507392672631:endpoint/huggingface-inference-eb
Status: InService


### 5. 模型测试

In [85]:
# Inference testing

import time
from sagemaker.huggingface import HuggingFaceModel
hfp = sagemaker.huggingface.model.HuggingFacePredictor('huggingface-inference-eb')
data = {
  "inputs": "你好。",
}
t0 = time.time()
for i in range(1):
    x = hfp.predict(data)
print(time.time()-t0)

#print(len(x[0][0]))
print(x)

0.0676882266998291
{'sentence_embeddings': [[-0.013831233605742455, -0.01270130556076765, -0.014087721705436707, 0.08026622235774994, 0.16525626182556152, 0.006540448404848576, -0.06130183860659599, 0.028956318274140358, 0.14638309180736542, 0.00560351088643074, 0.02524050697684288, -0.05012410134077072, -0.05577776953577995, -0.01011229120194912, 0.1492469310760498, -0.24108067154884338, 0.07875359803438187, 0.035032790154218674, -0.08368547260761261, -0.04624566435813904, 0.03269391879439354, -0.06078342720866203, 0.04449882358312607, -0.10587877035140991, -0.0344894714653492, 0.08417661488056183, -0.09343163669109344, 0.12138233333826065, 0.15460333228111267, 0.11100795120000839, 0.1600692868232727, -0.035302069038152695, 0.04129745811223984, 0.012123321183025837, 0.14945082366466522, -0.02292388118803501, 0.07285543531179428, 0.009303481318056583, -0.14735136926174164, -0.053448520600795746, 0.1369955837726593, -0.07428322732448578, -0.14402566850185394, 0.06494677811861038, -0.184

In [72]:
!aws sagemaker delete-endpoint --endpoint-name huggingface-inference-eb
!aws sagemaker delete-endpoint-config --endpoint-config-name huggingface-inference-eb
!aws sagemaker delete-model --model-name huggingface-inference-eb


An error occurred (ValidationException) when calling the DeleteModel operation: Could not find model "arn:aws-cn:sagemaker:cn-north-1:507392672631:model/huggingface-inference-eb".
